In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import ReLU, AvgPool2D
# from tensorflow.keras.layers import Input, DepthwiseConv2D
# from tensorflow.keras.layers import Conv2D, BatchNormalization
# from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
# from tensorflow.keras import Model

In [2]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5.csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images.shape)

1000
(1000, 320, 320, 9)


In [3]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

(800, 2)
(200, 2)
(800,)
(200,)


In [4]:
def create_cnn_alexnet():
    model = Sequential()
    model.add(Conv2D(filters = 96, input_shape = (320, 320, 9),  
                kernel_size = (11, 11), strides = (4, 4),  
                padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling  
    model.add(MaxPooling2D(pool_size = (2, 2), 
                strides = (2, 2), padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (11, 11),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 3rd Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 4th Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 5th Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # Flattening 
    model.add(Flatten()) 
    
    # 1st Dense Layer 
    model.add(Dense(4096, input_shape = (224*224*3, ))) 
    model.add(Activation('relu')) 
    # Add Dropout to prevent overfitting 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Dense Layer 
    model.add(Dense(4096)) 
    model.add(Activation('relu')) 
    # Add Dropout 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    return model

In [5]:
def create_cnn_resnet():
    model = Sequential()
    model.add(Conv2D(64,input_shape = (320,320,9), kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu'))
    model.add(AveragePooling2D(pool_size = 2))
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size = 2, padding = 'same'))
    model.add(Dropout(0.2))
    model.add(Flatten())

    return model

In [6]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def create_cnn_mobilenet():
    input = Input(shape = (320,320,9))
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # main part of the model
    x = mobilnet_block(x, filters = 64, strides = 1)
    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 1)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
   # output = Dense (units = 1000, activation = 'softmax')(x)
    output = Flatten()(x)
    model = Model(inputs=input, outputs=output)
    # model.summary()

    return model

In [7]:
alexnet = create_cnn_alexnet()
alexnetPredict = alexnet.predict(trainImagesX)
print(alexnetPredict.shape)



25/25 [==============================] - 14s 542ms/step
(800, 4096)


In [8]:
resnet = create_cnn_resnet()
resnetPredict = resnet.predict(trainImagesX)
print(resnetPredict.shape)

25/25 [==============================] - 25s 938ms/step
(800, 2048)


In [9]:
mobilenet = create_cnn_mobilenet()
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)

25/25 [==============================] - 13s 484ms/step
(800, 40720)


In [10]:
alexnetPredict = np.array(alexnetPredict)
resnetPredict = np.array(resnetPredict)
mobilenetPredict = np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(800,2)
trainData = concatenate([alexnetPredict,resnetPredict,mobilenetPredict,trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)

print(trainData.shape)

(800, 46866)


In [11]:
model = CascadeForestClassifier(n_estimators=50,random_state=1)
model.fit(trainData,trainy)

[2023-12-17 21:32:21.023] Start to fit the model:
[2023-12-17 21:32:21.023] Fitting cascade layer = 0 
[2023-12-17 21:48:44.859] layer = 0  | Val Acc = 98.625 % | Elapsed = 983.836 s
[2023-12-17 21:48:45.073] Fitting cascade layer = 1 
[2023-12-17 21:52:54.635] layer = 1  | Val Acc = 98.750 % | Elapsed = 249.561 s
[2023-12-17 21:52:54.646] Fitting cascade layer = 2 
[2023-12-17 21:56:46.808] layer = 2  | Val Acc = 98.875 % | Elapsed = 232.162 s
[2023-12-17 21:56:46.824] Fitting cascade layer = 3 
[2023-12-17 22:00:39.463] layer = 3  | Val Acc = 99.000 % | Elapsed = 232.639 s
[2023-12-17 22:00:39.471] Fitting cascade layer = 4 
[2023-12-17 22:04:42.423] layer = 4  | Val Acc = 98.875 % | Elapsed = 242.952 s
[2023-12-17 22:04:42.423] Early stopping counter: 1 out of 2
[2023-12-17 22:04:42.439] Fitting cascade layer = 5 
[2023-12-17 22:08:51.938] layer = 5  | Val Acc = 98.875 % | Elapsed = 249.499 s
[2023-12-17 22:08:51.938] Early stopping counter: 2 out of 2
[2023-12-17 22:08:51.938] Hand

In [12]:
# savemodel
# testAttrX
# testImagesX

alexnetPredictTest = np.array(alexnet.predict(testImagesX))
resnetPredictTest = np.array(resnet.predict(testImagesX))
mobilenetPredictTest = np.array(mobilenet.predict(testImagesX))
testAttrX = np.array(testAttrX)
print(alexnetPredictTest.shape)
print(resnetPredictTest.shape)
print(mobilenetPredictTest.shape)
print(testAttrX.shape)

testData = concatenate([alexnetPredictTest, resnetPredictTest, mobilenetPredictTest, testAttrX])
print(testData.shape)

7/7 [==============================] - 3s 416ms/step
(200, 4096)
(200, 2048)
(200, 40720)
(200, 2)
(200, 46866)


In [13]:
performance = model.predict(testData)
performance.round()
actual = []
for value in performance: 
    # print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

acutal = np.array(actual)
# print(actual)
# print(testy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

[2023-12-17 22:10:28.147] Start to evalute the model:
[2023-12-17 22:10:28.557] Evaluating cascade layer = 0 
[2023-12-17 22:10:28.969] Evaluating cascade layer = 1 
[2023-12-17 22:10:29.242] Evaluating cascade layer = 2 
[2023-12-17 22:10:29.486] Evaluating cascade layer = 3 
Precision: 1.000000
Recall: 1.000000
F1 score: 1.000000
